In [1]:
import matplotlib.pyplot as plt
%pylab inline
import numpy as np

Populating the interactive namespace from numpy and matplotlib


In [2]:
from getting_experiments_results import get_metrics_through_time, get_optimal_metrics_through_time

In [3]:
from data_keeper import get_data_keeper

In [4]:
EXPERIMENTS_RESULTS_PATH = "final_res"

In [5]:
def savitzky_golay(y, window_size, order, deriv=0, rate=1):
    r"""Smooth (and optionally differentiate) data with a Savitzky-Golay filter.
    The Savitzky-Golay filter removes high frequency noise from data.
    It has the advantage of preserving the original shape and
    features of the signal better than other types of filtering
    approaches, such as moving averages techniques.
    Parameters
    ----------
    y : array_like, shape (N,)
        the values of the time history of the signal.
    window_size : int
        the length of the window. Must be an odd integer number.
    order : int
        the order of the polynomial used in the filtering.
        Must be less then `window_size` - 1.
    deriv: int
        the order of the derivative to compute (default = 0 means only smoothing)
    Returns
    -------
    ys : ndarray, shape (N)
        the smoothed signal (or it's n-th derivative).
    Notes
    -----
    The Savitzky-Golay is a type of low-pass filter, particularly
    suited for smoothing noisy data. The main idea behind this
    approach is to make for each point a least-square fit with a
    polynomial of high order over a odd-sized window centered at
    the point.
    Examples
    --------
    t = np.linspace(-4, 4, 500)
    y = np.exp( -t**2 ) + np.random.normal(0, 0.05, t.shape)
    ysg = savitzky_golay(y, window_size=31, order=4)
    import matplotlib.pyplot as plt
    plt.plot(t, y, label='Noisy signal')
    plt.plot(t, np.exp(-t**2), 'k', lw=1.5, label='Original signal')
    plt.plot(t, ysg, 'r', label='Filtered signal')
    plt.legend()
    plt.show()
    References
    ----------
    .. [1] A. Savitzky, M. J. E. Golay, Smoothing and Differentiation of
       Data by Simplified Least Squares Procedures. Analytical
       Chemistry, 1964, 36 (8), pp 1627-1639.
    .. [2] Numerical Recipes 3rd Edition: The Art of Scientific Computing
       W.H. Press, S.A. Teukolsky, W.T. Vetterling, B.P. Flannery
       Cambridge University Press ISBN-13: 9780521880688
    """
    import numpy as np
    from math import factorial
    
    try:
        window_size = np.abs(np.int(window_size))
        order = np.abs(np.int(order))
    except ValueError, msg:
        raise ValueError("window_size and order have to be of type int")
    if window_size % 2 != 1 or window_size < 1:
        raise TypeError("window_size size must be a positive odd number")
    if window_size < order + 2:
        raise TypeError("window_size is too small for the polynomials order")
    order_range = range(order+1)
    half_window = (window_size -1) // 2
    # precompute coefficients
    b = np.mat([[k**i for i in order_range] for k in range(-half_window, half_window+1)])
    m = np.linalg.pinv(b).A[deriv] * rate**deriv * factorial(deriv)
    # pad the signal at the extremes with
    # values taken from the signal itself
    firstvals = y[0] - np.abs( y[1:half_window+1][::-1] - y[0] )
    lastvals = y[-1] + np.abs(y[-half_window-1:-1][::-1] - y[-1])
    y = np.concatenate((firstvals, y, lastvals))
    return np.convolve( m[::-1], y, mode='valid')

In [6]:
import sys
from concurrent.futures import ProcessPoolExecutor
def get_drug_result(drug):
    conv_plot_done = not (drug == get_data_keeper().get_possible_second_level_drugs()[0])
    result = dict()
    drug_result = dict()
    result[drug] = drug_result
    drug_result['F1'] = dict()
    drug_result['accuracy'] = dict()
    drug_result['0/0'] = dict()
    drug_result['0/1'] = dict()
    drug_result['1/0'] = dict()
    drug_result['1/1'] = dict()
    for experiment_name in ['model', 'selector_model', 'frn_model', 'extender_robust_model', 'extender_selector_model','extender_frn_model']:
        local_results = dict()
        result[drug][experiment_name] = local_results
        metrics = get_metrics_through_time(
            experiment_name=experiment_name,
            drug=drug,
            experiments_results_path=EXPERIMENTS_RESULTS_PATH,
        )
        optimal_metrics = get_optimal_metrics_through_time(
            experiment_name=experiment_name,
            drug=drug,
            experiments_results_path=EXPERIMENTS_RESULTS_PATH,
        )
        if not conv_plot_done:
            local_results['convergence_plot_x'] = [el[0] - metrics[0][0] for el in metrics]
            local_results['convergence_plot_y'] = savitzky_golay([np.mean([el1['full_metrics']['accuracy'] for el1 in el[1]]) for el in metrics], 21, 3)
            conv_plot_done = True
        local_results['best_accuracy_plot_cv_x'] = [el[0] - optimal_metrics[0][0] for el in optimal_metrics]
        local_results['best_accuracy_plot_cv_y'] = [np.mean([el1['full_metrics']['accuracy'] for el1 in el[1]]) for el in optimal_metrics] 
        local_results['best_accuracy_plot_test_x'] = [el[0] - optimal_metrics[0][0] for el in optimal_metrics]
        local_results['best_accuracy_plot_test_y'] = [el[2]['accuracy'] for el in optimal_metrics]
        result_metrics = optimal_metrics[-1][2]
        drug_result['F1'][experiment_name] = result_metrics['f1_score']
        drug_result['accuracy'][experiment_name] = result_metrics['accuracy']
        drug_result['0/0'][experiment_name] = result_metrics['confusion_matrix'][0][0]
        drug_result['0/1'][experiment_name] = result_metrics['confusion_matrix'][0][1]
        drug_result['1/0'][experiment_name] = result_metrics['confusion_matrix'][1][0]
        drug_result['1/1'][experiment_name] = result_metrics['confusion_matrix'][1][1]
        print "{} {} done".format(drug, experiment_name)
        sys.stdout.flush()
    return result
result = dict()
    
with ProcessPoolExecutor(max_workers=6) as e:
    for res in e.map(get_drug_result, get_data_keeper().get_possible_second_level_drugs()):
        result.update(res)

/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


CAPR: Capreomycin  model done
AMIK: Amikacin model done
OFLO: Ofloxacin  model done
ETHI: Ethionamide/ Prothionamide  model done
PARA: Para-aminosalicyclic acid  model done


/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


AMIK: Amikacin selector_model done
CAPR: Capreomycin  selector_model done
ETHI: Ethionamide/ Prothionamide  selector_model done
PARA: Para-aminosalicyclic acid  selector_model done
OFLO: Ofloxacin  selector_model done
AMIK: Amikacin frn_model done
CAPR: Capreomycin  frn_model done
AMIK: Amikacin extender_robust_model done
CAPR: Capreomycin  extender_robust_model done
CAPR: Capreomycin  extender_selector_model done
AMIK: Amikacin extender_selector_model done
CAPR: Capreomycin  extender_frn_model done
ETHI: Ethionamide/ Prothionamide  frn_model done
OFLO: Ofloxacin  frn_model done
AMIK: Amikacin extender_frn_model done
PARA: Para-aminosalicyclic acid  frn_model done
OFLO: Ofloxacin  extender_robust_model done
OFLO: Ofloxacin  extender_selector_model done
OFLO: Ofloxacin  extender_frn_model done
PARA: Para-aminosalicyclic acid  extender_robust_model done
ETHI: Ethionamide/ Prothionamide  extender_robust_model done
PARA: Para-aminosalicyclic acid  extender_selector_model done
ETHI: Ethiona

In [10]:
import pandas as pd
d = {
    'model': '4.4.1',
    'selector_model': '4.4.2',
    'frn_model': '4.4.3',
    'extender_robust_model': '4.4.4',
    'extender_selector_model': '4.4.5',
    'extender_frn_model': '4.4.6',
}
result_refactored = {key: {(k, d[v1]): v2 for k, v in value.iteritems() for v1, v2 in v.iteritems() if 'plot' not in v1} for key, value in result.iteritems()}
df = pd.DataFrame.from_dict(result_refactored)
df.index = pd.MultiIndex.from_tuples(df.index)
df.to_excel("a.xls")

In [11]:
list(df.T.columns.values)

[('0/0', '4.4.1'),
 ('0/0', '4.4.2'),
 ('0/0', '4.4.3'),
 ('0/0', '4.4.4'),
 ('0/0', '4.4.5'),
 ('0/0', '4.4.6'),
 ('0/1', '4.4.1'),
 ('0/1', '4.4.2'),
 ('0/1', '4.4.3'),
 ('0/1', '4.4.4'),
 ('0/1', '4.4.5'),
 ('0/1', '4.4.6'),
 ('1/0', '4.4.1'),
 ('1/0', '4.4.2'),
 ('1/0', '4.4.3'),
 ('1/0', '4.4.4'),
 ('1/0', '4.4.5'),
 ('1/0', '4.4.6'),
 ('1/1', '4.4.1'),
 ('1/1', '4.4.2'),
 ('1/1', '4.4.3'),
 ('1/1', '4.4.4'),
 ('1/1', '4.4.5'),
 ('1/1', '4.4.6'),
 ('F1', '4.4.1'),
 ('F1', '4.4.2'),
 ('F1', '4.4.3'),
 ('F1', '4.4.4'),
 ('F1', '4.4.5'),
 ('F1', '4.4.6'),
 ('accuracy', '4.4.1'),
 ('accuracy', '4.4.2'),
 ('accuracy', '4.4.3'),
 ('accuracy', '4.4.4'),
 ('accuracy', '4.4.5'),
 ('accuracy', '4.4.6')]